In [31]:
import torch
from torch import nn

In [45]:
lenet = nn.Sequential(nn.Conv2d(1,6, kernel_size=5, padding=2), nn.Sigmoid(),
                    nn.AvgPool2d(kernel_size=2,stride=2),
                    nn.Conv2d(6,16,kernel_size=5), nn.Sigmoid(),
                    nn.AvgPool2d(kernel_size=2, stride=2),
                    nn.Flatten(),
                    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
                    nn.Linear(120,84), nn.Sigmoid(), nn.Linear(84,10)
                    )

In [33]:
X = torch.randn((1,1,28,28))

In [46]:
for layer in lenet:
    X = layer(X)
    
    print(f"Shape of {layer.__class__.__name__} : {X.shape}")

Shape of Conv2d : torch.Size([256, 6, 28, 28])
Shape of Sigmoid : torch.Size([256, 6, 28, 28])
Shape of AvgPool2d : torch.Size([256, 6, 14, 14])
Shape of Conv2d : torch.Size([256, 16, 10, 10])
Shape of Sigmoid : torch.Size([256, 16, 10, 10])
Shape of AvgPool2d : torch.Size([256, 16, 5, 5])
Shape of Flatten : torch.Size([256, 400])
Shape of Linear : torch.Size([256, 120])
Shape of Sigmoid : torch.Size([256, 120])
Shape of Linear : torch.Size([256, 84])
Shape of Sigmoid : torch.Size([256, 84])
Shape of Linear : torch.Size([256, 10])


In [36]:
batch_size = 256


import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [37]:
my_transforms = transforms.ToTensor()

train_dataset = datasets.FashionMNIST(root="../data", download=False, train=True, transform=my_transforms)
test_dataset = datasets.FashionMNIST(root="../data", download=False, train=False, transform=my_transforms)

In [38]:
X,y = train_dataset[0]
X.shape,y

(torch.Size([1, 28, 28]), 9)

In [41]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [42]:
for X, y in train_dataloader:
    print(X, y)
    break

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0157,  ..

Since the full dataset is in the main memory, we need to copy it to the
GPU memory before the model uses GPU to compute with the dataset

In [43]:
def accuracy(y_hat, y):
    return (y_hat.argmax(1)==y).sum()

In [68]:
def evaluate_accuracy_gpu(net, data_iter):
    if isinstance(net, nn.Module):
        net.eval()
        
        device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
    
    acc_arr = 0
    total_number = 0
    
    for X, y in data_iter:
        X = X.to(device).float()
        y = y.to(device).float()
        net = net.to(device)
        y_hat = net(X)
        acc_arr += accuracy(y_hat, y)
        total_number += y.numel()
    
    return acc_arr/total_number
        
        

In [69]:
#before training
evaluate_accuracy_gpu(lenet, test_dataloader)

tensor(0.1000, device='cuda:0')

In [28]:
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    
    net.apply(init_weights)
    
    print(f'training on {device}')
    
    net = net.to(device)
    
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    
    train_loss = []
    train_acc = []
    test_acc = []
    
    for epoch in range(num_epochs):
        current_l = 0
        for i, (X, y) in enumerate(train_iter):
            X = X.to(device).float()
            y = y.to(device).float()
        
            y_hat = net(X)
        
            l = loss(y_hat, y)
            optimizer.zero_grad()
        
            l.backward()
        
            optimizer.step()
            
            current_l += l
            
        print(f'For epoch {epoch} the loss {l}')
        
        
        

In [30]:
lr, num_epochs = 0.9, 10

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

train_ch6(net, train_iter=train_dataloader, test_iter=test_dataloader, num_epochs=num_epochs, lr=0.9, device=device)

training on cuda


RuntimeError: DataLoader worker (pid(s) 8232, 15036) exited unexpectedly